In [145]:
#ライブラリのインポート
import tkinter as tk
import cv2
import PIL.Image, PIL.ImageTk
from PIL import Image, ImageTk, ImageOps
from tkinter import messagebox
import json
import os
import re

class Application(tk.Frame):
    
    ori_data_path = "./data/ori_data/"
    files_path = os.listdir(ori_data_path + 'drama_image')
    files = sorted(files_path, key=lambda s: int(re.search('\d+', s).group()))
    
    image_id = []
    
    bg_color = ["pink","white","white","pink","pink","lightgreen","lightgreen","orange","orange","lime","lime","pink","yellow","yellow","pink","lavender","lavender"]
    
    object = ["歩行者", "四輪車", "オートバイ", "自転車"]
    
    item = ["歩行者が歩きスマホなどで注意不足状態である",
            "対向車両がセンターラインよりを走行している",
            "対向車両がセンターラインよりを走行するかもしれない",
            "駐車車両が発進するかもしれない",
            "駐車車両の扉が開くかもしれない",
            "対向右折車が急いで右折している",
            "対向右折車が急いで右折するかもしれない",
            "対象がふらついている",
            "対象がふらつくかもしれない",
            "対象が自車進路に侵入している",
            "対象が自車進路に侵入するかもしれない",
            "対象が自車の存在に気づいていない",
            "対象が急停止している",
            "対象が急停止するかもしれない",
            "対象が他の物体と接触するかもしれない",
            "対象が進路変更している",
            "対象が進路変更するかもしれない"]
    

    
    rect_list = []
    rect_id_list = []
    label_list = []
    txt_list = []
    check_list = []
    radio_list = []
    
    button_list = []
    risk_label_list = []
    
    risk_priority_list = []
    
    frame_list = []
    
    label_pos_list = []
    button_pos_list = []
    text_pos_list = []
    check_pos_list = []
    radio_pos_list = []
    

    risk_label_pos_list = []
    
    risk_priority_pos_list= []


    frame_pos_list = []
    
    all_check_var_list = []
    all_radio_var_list = []
    check_var_list = []
    radio_var_list = []
    

    check_var_dict = {}
    txt_var_list = []
    
    
    box_start_list = []
    box_end_list = []
    
    region_json_data = []
    image_json_data = []
    class_list = []
    
    counter_box = -1
    counter_img = 1
    region_id = 0
    remove_counter = 0
    one_space = 300
    
    
    def __init__(self, master=None):
        super().__init__(master)
        self.master.geometry("2000x1000")#ウィンドウサイズ
        #self.master.attributes('-fullscreen', False)
        self.master.title("OpenCV 動画表示")#タイトル設定
        
        if os.path.exists(self.ori_data_path + 'region_danger_info_data.json') and os.path.exists(self.ori_data_path + 'img_info_data.json'):
            
            region_json_open = open(self.ori_data_path + 'region_danger_info_data.json', 'r')
            image_json_open = open(self.ori_data_path + 'img_info_data.json', 'r')
            region_json_load = json.load(region_json_open)
            image_json_load = json.load(image_json_open)
        
            if region_json_load == None or image_json_load == None:
                #print("a")
                pass
            else:
                for i,one_region_data in enumerate(region_json_load):
                    self.region_json_data.append(one_region_data)#既存の領域データを変数に格納
                    
                    if i == len(region_json_load)-1:
                        self.counter_img = self.region_json_data[-1]["id"] + 1
                        self.region_id = self.region_json_data[-1]["regions"][-1]['region_id']
                    
                    
                for i,one_image_data in enumerate(image_json_load):
                    self.image_json_data.append(one_image_data)#既存の画像データを変数に格納
        
        #動画ファイルの読み込み
        self.cap = cv2.VideoCapture('C:/Users/naoki/anaconda_work/Faster-R-CNN-pytorch-main/data/ori_data/drama_clip/{}/clip.mp4'.format(self.counter_img - 1))
        
        #プロパティ
        self.width = self.cap.get( cv2.CAP_PROP_FRAME_WIDTH)
        self.height = self.cap.get( cv2.CAP_PROP_FRAME_HEIGHT)
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.frame_num = self.cap.get(cv2.CAP_PROP_FRAME_COUNT)
        self.time = self.frame_num / self.fps
        #print(self.fps,self.frame_num)
        self.create_frame()
        self.create_widget()
        self.button_conf()
        self.delay = 10
        self.play_video()

    def create_frame(self):
        #フレームの作成
        self.frame1 = tk.Frame(self.master, width=1000, height=740, bg="#E6E6E6")
        self.frame1.place(x=20, y=20)
        self.frame2 = tk.Frame(self.master, width=640, height=925, relief=tk.RAISED, bd=5, bg="lightgrey")
        self.frame2.place(x=1025, y=20)
        self.frame3 = tk.Frame(self.master, width=600, height=20, relief=tk.RAISED, bd=3, bg="lightgrey")
        self.frame3.place(x=20, y=760)
        self.frame4 = tk.Frame(self.master, width=250, height=100, relief=tk.RAISED, bd=3, bg="lightgrey")
        self.frame4.place(x=750, y=875)
        self.frame5 = tk.Frame(self.master, width=500, height=40, relief=tk.RAISED, bd=3, bg="lightgrey")
        self.frame5.place(x=1100, y=935)
        
        # Canvasの作成
        self.canvas = tk.Canvas(self.frame1, width=1000, height=740,)
        self.anno_canvas = tk.Canvas(self.frame2, width=640, height=925,)
        # Canvasを配置
        self.canvas.pack()
        self.anno_canvas.pack()
        
    def create_widget(self):
        #Labelの生成
        l_property = tk.Label(self.frame3,text="プロパティ", relief="flat")
        l_property.grid(row=0, column=0, sticky = tk.W)
        l_id = tk.Label(self.frame3,text="image_id:{}".format(self.counter_img), relief="flat")
        l_id.grid(row=0, column=1, sticky = tk.W)
        l_width = tk.Label(self.frame3,text="width: "+str(int(self.width)), relief="flat")
        l_width.grid(row=0, column=2, sticky = tk.W)
        l_height = tk.Label(self.frame3,text="height: "+str(int(self.height)), relief="flat")
        l_height.grid(row=0, column=3, sticky = tk.W)
        l_fps = tk.Label(self.frame3,text="fps: "+str(int(self.fps)), relief="flat")
        l_fps.grid(row=0, column=4, sticky = tk.W)
        l_time = tk.Label(self.frame3,text="time: "+str(int(self.time))+"sec", relief="flat")
        l_time.grid(row=0, column=5, sticky = tk.W)
        
    def button_conf(self):
        None_remove_list = [i for i in range(len(self.label_list)) if self.label_list[i] is not None]
        #次の動画へ、もう一度再生
        next_button = tk.Button(self.frame4, width=10, height = 5, text='NEXT', command=self.btn_click_next)#, command=self.btn_click_next)
        next_button.place(x=140, y=5)
        re_button = tk.Button(self.frame4, width=10, height = 5, text='もう一度再生', command=self.btn_click_re)#, command=self.btn_click_next)
        re_button.place(x=20, y=5)
        
        #ページ番号のボタン
        self.one_button = tk.Button(self.frame5, width=1, height = 1, text='1',command=lambda: self.button_clicked(1))#, command=self.btn_click_next)
        self.one_button.place(x=5, y=5)
        self.one_button.config(state=tk.DISABLED)
        self.button_state = 1
        self.two_button = tk.Button(self.frame5, width=1, height = 1, text='2',command=lambda: self.button_clicked(2))#, command=self.btn_click_next)
        self.two_button.place(x=25, y=5)
        self.two_button.config(state=tk.DISABLED)
          
    def button_clicked(self, button_num):
    # ボタンの状態を変更する
        if button_num == 1:
            self.button_state = 1
            self.one_button.config(state=tk.DISABLED)  # ボタン1を無効化
            self.two_button.config(state=tk.NORMAL)
            
            self.data_hidden(1)
            
        if button_num == 2:
            self.button_state = 2
            self.two_button.config(state=tk.DISABLED)  # ボタン2を無効化
            self.one_button.config(state=tk.NORMAL)
            
            self.data_hidden(2)
               
    def play_video(self):
        #動画をリサイズしてキャンバスに表示
        
        ret, frame = self.cap.read()
        if ret == False:
            self.image_view()
            return
        
        self.current_frame = self.cap.get(cv2.CAP_PROP_POS_FRAMES)
        l_time = tk.Label(self.frame3,text="frame: "+str(int(self.current_frame)), relief="flat")
        l_time.grid(row=0, column=5, sticky = tk.W)

        frame = cv2.resize(frame, dsize=(1000, 740))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
        self.canvas.create_image(0,0, image= self.photo, anchor = tk.NW)
        #alter関数 10ms後に関数を実行
        self.master.after(self.delay, self.play_video)
        
    def image_view(self):#画像の表示
        
        self.image = Image.open("C:/Users/naoki/anaconda_work/Faster-R-CNN-pytorch-main/data/ori_data/drama_image/{}.jpg".format(self.counter_img))
        self.tk_img = ImageTk.PhotoImage(self.image)
        self.img_width, self.img_height = self.image.size
        self.canvas.create_image(0,0, image= self.tk_img, anchor = tk.NW)
        for i in range(len(self.rect_list)):
            if self.box_start_list[i] != None:
                self.canvas.create_rectangle(self.box_start_list[i][0], self.box_start_list[i][1], self.box_end_list[i][0], self.box_end_list[i][1], 
                                         outline = 'red', width = 5, tag="rect_{}".format(i))#四角を描画
        for i in range(len(self.rect_id_list)):
            if self.rect_id_list[i] != None:
                self.rect_id_list[i].place(x=self.box_start_list[i][0], y=self.box_start_list[i][1])
        self._createCanvasBinding()#操作開始self.box_start_list
        
    def _createCanvasBinding(self):
        
        """
        
        マウスクリックなどのイベントを識別して、イベントに応じた関数を呼び出す
        
        """
        
        self.canvas.bind( "<Button-1>", self.startRect )
        self.canvas.bind( "<ButtonRelease-1>", self.stopRect )
        self.canvas.bind( "<B1-Motion>", self.movingRect )

    def unbind(self):
        
        self.canvas.unbind("<Button-1>")
        self.canvas.unbind("<ButtonRelease-1>")
        self.canvas.unbind("<B1-Motion>")
    
    def btn_click(self, create_numbers):
        
        """
        
        ボックスに付与されている "取り消し" ボタンが押されたときの処理
        
        args
        ---------------
        
        create_numbers
            何番目に作成されたボックスかを表すID
        
        """
        
        def x():
            self.remove_counter += 1
            self.canvas.delete(self.rect_list[create_numbers])
            self.rect_list[create_numbers] = None
            self.rect_id_list[create_numbers].destroy()
            self.rect_id_list[create_numbers] = None
            self.label_list[create_numbers].destroy()
            self.label_list[create_numbers] = None
            self.risk_label_list[create_numbers].destroy()
            self.risk_label_list[create_numbers] = None
            
            self.risk_priority_list[create_numbers].destroy()
            self.risk_priority_list[create_numbers] = None

            self.anno_canvas.delete("frame_{}".format(create_numbers))
            
            
            for i in range(len(self.item)):
                
                self.check_list[create_numbers][i].destroy()
                self.check_list[create_numbers][i] = None
            
            for i in range(len(self.object)):
                
                self.radio_list[create_numbers][i].destroy()
                self.radio_list[create_numbers][i] = None
            
            self.all_check_var_list[create_numbers] = None
            self.all_radio_var_list[create_numbers] = None
            
            self.txt_list[create_numbers].destroy()
            self.txt_list[create_numbers] = None
            self.button_list[create_numbers].destroy()
            self.button_list[create_numbers] = None
            
            self.canvas.delete("rect_{}".format(create_numbers))
            self.box_start_list[create_numbers] = None
            self.box_end_list[create_numbers] = None
            
            self.check_var_list[create_numbers] = None
            self.radio_var_list[create_numbers] = None
            self.txt_var_list[create_numbers] = None
            
            None_remove_list = [i for i in range(len(self.label_list)) if self.label_list[i] is not None]
            num = 1
            if self.button_state == 2:
                num = len(None_remove_list) // 3 * 3 + 1

            #print(len(None_remove_list))
            for i in range(create_numbers,len(self.button_list)):
                
                if self.button_list[i] != None:
                    
                    self.button_list[i].place(x=self.button_pos_list[i][0],y=self.button_pos_list[i][1]-self.one_space * num)
                    self.button_pos_list[i][1] -= self.one_space
                    
                    self.txt_list[i].place(x=self.text_pos_list[i][0],y=self.text_pos_list[i][1]-self.one_space * num)
                    self.text_pos_list[i][1] -= self.one_space
                    
                    self.label_list[i].place(x=self.label_pos_list[i][0],y=self.label_pos_list[i][1]-self.one_space * num)
                    self.label_pos_list[i][1] -= self.one_space
                    
                    self.risk_label_list[i].place(x=self.risk_label_pos_list[i][0],y=self.risk_label_pos_list[i][1]-self.one_space * num)
                    self.risk_label_pos_list[i][1] -= self.one_space

                    self.risk_priority_list[i].place(x=self.risk_priority_pos_list[i][0],y=self.risk_priority_pos_list[i][1]-self.one_space * num)
                    self.risk_priority_pos_list[i][1] -= self.one_space
                    
                    self.anno_canvas.itemconfigure("frame_{}".format(i), state='normal')
                    self.anno_canvas.move("frame_{}".format(i), 0, -self.one_space)
                    self.frame_pos_list[i][1] -= self.one_space
                    
                    
                    for l in range(len(self.item)):
                        #print(self.check_list[i][l])
                        
                        self.check_list[i][l].place(x=self.check_pos_list[i][l][0],y=self.check_pos_list[i][l][1]-self.one_space * num)
                        self.check_pos_list[i][l][1] -= self.one_space
                    
                    for l in range(len(self.object)):
                        #print(self.check_list[i][l])
                        
                        self.radio_list[i][l].place(x=self.radio_pos_list[i][l][0],y=self.radio_pos_list[i][l][1]-self.one_space * num)
                        self.radio_pos_list[i][l][1] -= self.one_space
                    
                if len(None_remove_list) >= 4 and self.button_state == 1:
                    for i in None_remove_list[3:]:
                        #print(i)
                        self.label_list[i].place_forget()
                        self.risk_label_list[i].place_forget()
                        self.risk_priority_list[i].place_forget()
                        self.txt_list[i].place_forget()
                        self.button_list[i].place_forget()
                        self.anno_canvas.itemconfigure("frame_{}".format(i), state='hidden')
                
                        for item in range(len(self.item)):
                            self.check_list[i][item].place_forget()
                        for item in range(len(self.object)):
                            self.radio_list[i][item].place_forget()
            if len(None_remove_list) < 6:
                self._createCanvasBinding()
            if len(None_remove_list) == 3:
                print("a")
                self.button_state = 1
                self.one_button.config(state=tk.DISABLED)  # ボタン1を無効化
                self.two_button.config(state=tk.NORMAL)
                self.data_hidden(1)
            if len(None_remove_list) < 4:
                self.two_button.config(state=tk.DISABLED)
            if len(None_remove_list) == 0:
                self.two_button.config(state=tk.DISABLED)
        return x
    
    def startRect(self, event):
        
        """
        
        マウスを左クリックされたときに呼び出される関数
        
        args
        ---------------
        
        event : tkinter.Eventクラス
            左クリックしたイベントの情報が保持されたクラス
            event.x はクリックされたx座標
            event.y はクリックされたy座標
        
        """
    
        
        self.counter_box += 1
        
        self.rectx0 = self.canvas.canvasx(event.x)
        self.recty0 = self.canvas.canvasy(event.y) 
        
        self.rectid = self.canvas.create_rectangle(
            self.rectx0, self.recty0, self.rectx0, self.recty0, outline = 'red', width = 5)
        
        self.rect_list.append(self.rectid)
    
    def movingRect(self, event):
        
        """
        
        マウスが左クリックされた状態でのドラッグ動作の情報を取得し、情報をウィンドウに表示する。
        
        args
        ---------------
        
        event : tkinter.Eventクラス
            ドラッグ状態の情報が保持されたクラス
            event.x は現在のx座標
            event.y は現在のy座標
        
        """
        
        self.rectx1 = self.canvas.canvasx(event.x)
        self.recty1 = self.canvas.canvasy(event.y)
        
        self.canvas.coords(self.rectid, self.rectx0, self.recty0,
                      self.rectx1, self.recty1)
        
    def stopRect(self, event):
        
        """
        
        左クリックが離されたときの情報を取得し、各種処理を行う関数
        
        args
        ---------------
        
        event : tkinter.Eventクラス
            左クリックを離すというイベントの情報が保持されたクラス
            event.x は左クリックが離されたときのx座標
            event.y は左クリックが離されたときのy座標
        
        """
        
        None_remove_list = [i for i in range(len(self.label_list)) if self.label_list[i] is not None]
        
        if self.button_state == 2:
            num = len(None_remove_list) // 3 * 3 
        else:
            num = 0
        #print(self.counter_box)
        #print(self.remove_counter)
        self.rectx1 = self.canvas.canvasx(event.x)
        self.recty1 = self.canvas.canvasy(event.y)
        
        self.canvas.coords(self.rectid, self.rectx0, self.recty0,
                      self.rectx1, self.recty1)
        
        self.box_start_list.append([self.rectx0, self.recty0])
        self.box_end_list.append([self.rectx1, self.recty1])
        
        self.txt_var_list.append(tk.StringVar())
        self.txt_var_list[self.counter_box].set("")
        
        self.rect_id_list.append(tk.Label(self.frame1, text="ID : {}".format(self.counter_box), width=5))
        self.rect_id_list[self.counter_box].place(x=self.rectx0, y=self.recty0)
        
        one_box_check = []
        one_box_radio = []

        self.check_var_dict = {}

        for i in range(len(self.item)):
            self.check_var_dict[i]=tk.BooleanVar()
            self.check_var_dict[i].set(False)
        self.check_var_list.append(self.check_var_dict)
        
        
        for i in range(len(self.item)):
            one_box_check.append(tk.Checkbutton(self.frame2,variable=self.check_var_list[self.counter_box][i],
                   text=self.item[i], bg = self.bg_color[i]))
        self.check_list.append(one_box_check)
        
        self.all_check_var_list.append(self.check_var_dict)
        
        radio_var = tk.IntVar()
        radio_var.set(-1)
        self.radio_var_list.append(radio_var)
        
        for i in range(len(self.object)):
            one_box_radio.append(tk.Radiobutton(self.frame2,variable=self.radio_var_list[self.counter_box],
                   text=self.object[i],value=i, command=self.disable_checkboxes(self.counter_box)))
        self.radio_list.append(one_box_radio)
        
        
        self.all_radio_var_list=self.radio_var_list
        

        one_c_pos = []
        
        for i in range(len(self.item)):
            c_x, c_y = 20  , 70 + i * 25 + self.counter_box * self.one_space
            if i >= 9:
                c_x, c_y = 300  , 70 + (i-9) * 25 + self.counter_box * self.one_space
                
            one_c_pos.append([c_x, c_y - self.remove_counter * self.one_space])
            self.check_list[self.counter_box][i].place(x=c_x, y=c_y - self.remove_counter * self.one_space - num * self.one_space)
        self.check_pos_list.append(one_c_pos)
        
        one_object_pos = []
        for i in range(len(self.object)):
            object_x, object_y = 80 + i * 80, 10 + self.counter_box * self.one_space
            one_object_pos.append([object_x, object_y - self.remove_counter * self.one_space])
            self.radio_list[self.counter_box][i].place(x=object_x, y=object_y - self.remove_counter * self.one_space - num * self.one_space)
        self.radio_pos_list.append(one_object_pos)
        
        
        l_x , l_y = 10, 10  + self.counter_box * self.one_space
        self.label_pos_list.append([l_x , l_y - self.remove_counter * self.one_space])
        self.label_list.append(tk.Label(self.frame2, text="領域ID : {}".format(self.counter_box), width=8))
        self.label_list[self.counter_box].place(x=l_x, y=l_y - self.remove_counter * self.one_space - num * self.one_space)
        
        t_x , t_y = 400, 40 + self.counter_box * self.one_space
        self.text_pos_list.append([t_x ,t_y - self.remove_counter * self.one_space])
        self.txt_list.append(tk.Entry(self.frame2, width=20,textvariable = self.txt_var_list[self.counter_box]))
        self.txt_list[self.counter_box].place(x=t_x, y=t_y - self.remove_counter * self.one_space - num * self.one_space)
        
        b_x ,b_y = 550, 10  + self.counter_box * self.one_space
        self.button_pos_list.append([b_x ,b_y - self.remove_counter * self.one_space])
        self.button_list.append(tk.Button(self.frame2,width=7,text='取り消し', background='red', command=self.btn_click(self.counter_box)))
        self.button_list[self.counter_box].place(x=b_x, y=b_y - self.remove_counter * self.one_space - num * self.one_space)
        
        r_x , r_y = 20, 40  + self.counter_box * self.one_space
        self.risk_label_pos_list.append([r_x , r_y - self.remove_counter * self.one_space])
        self.risk_label_list.append(tk.Label(self.frame2, text="危険要因",background='lightgreen' ,width=8))
        self.risk_label_list[self.counter_box].place(x=r_x, y=r_y - self.remove_counter * self.one_space - num * self.one_space)
        
        
        
        priority_label_x , priority_label_y = 330, 40  + self.counter_box * self.one_space
        self.risk_priority_pos_list.append([priority_label_x , priority_label_y - self.remove_counter * self.one_space])
        self.risk_priority_list.append(tk.Label(self.frame2, text="その他",background='orange', width=8))
        self.risk_priority_list[self.counter_box].place(x=priority_label_x, y=priority_label_y - self.remove_counter * self.one_space - num * self.one_space)
        

        s_x , s_y = 5, 20  + self.counter_box * self.one_space
        self.frame_pos_list.append([s_x , s_y - self.remove_counter * self.one_space])
        self.frame_list.append(self.anno_canvas.create_rectangle(s_x, s_y- self.remove_counter * self.one_space - num * self.one_space, 
                                                                 s_x + 580, s_y + self.one_space - 20 - self.remove_counter * self.one_space - num * self.one_space, 
                                                                 fill = "pink", tag = "frame_{}".format(self.counter_box)))

        if self.button_state == 1 and len(None_remove_list) >= 3:
            self.label_list[self.counter_box].place_forget()
            self.risk_label_list[self.counter_box].place_forget()
            self.risk_priority_list[self.counter_box].place_forget()
            self.txt_list[self.counter_box].place_forget()
            self.button_list[self.counter_box].place_forget()
            self.anno_canvas.itemconfigure("frame_{}".format(self.counter_box), state='hidden')
                
            for item in range(len(self.item)):
                self.check_list[self.counter_box][item].place_forget()
            for item in range(len(self.object)):
                self.radio_list[self.counter_box][item].place_forget()
        if len(None_remove_list) <= 2:
            self.two_button.config(state=tk.DISABLED)
        else:
            self.two_button.config(state=tk.NORMAL)
        
        if len(None_remove_list) == 5:
            self.unbind()
            messagebox.showerror('上限', '最大選択数に達しました')
        else:
            self._createCanvasBinding()
        
        if len(None_remove_list) == 3:
            self.button_state = 2
            self.two_button.config(state=tk.DISABLED)  # ボタン2を無効化
            self.one_button.config(state=tk.NORMAL)
            self.data_hidden(2)
            
    def btn_click_next(self):
        
        record_obj_data = list(filter(None, self.all_radio_var_list))
        record_risk_data = list(filter(None, self.all_check_var_list))
        
        for one_data_obj in record_obj_data:
            check = one_data_obj.get()
            
        if check == -1:
            messagebox.showerror('未入力', '物体の種類が入力されていません')
            return
        
        for one_data_risk in record_risk_data:
            risk_check = []
            for i in range(len(self.item)):
                risk_check.append(one_data_risk[i].get())
            if (risk_check[1] == 1 and risk_check[2]) or (risk_check[5] == 1 and risk_check[6]) or (risk_check[7] == 1 and risk_check[8]) or (risk_check[9] == 1 and risk_check[10]) or (risk_check[12] == 1 and risk_check[13]) or (risk_check[15] == 1 and risk_check[16]):
                messagebox.showerror('矛盾選択', '重複選択しています')
                return
        
        self.image_id = self.files[self.counter_img].split(".")[0]
        self.image_id = int(self.image_id) - 1
        data_exist_list = list(filter(None, self.box_start_list))
        
        l_id = tk.Label(self.frame3,text="image_id:{}".format(self.image_id+1), relief="flat")
        l_id.grid(row=0, column=1, sticky = tk.W)#表示しているimage_idの更新
            
        
        if data_exist_list != []:
            self.save_json(self.box_start_list, self.box_end_list, 
                        self.txt_var_list, self.all_check_var_list,
                        self.all_radio_var_list,
                        self.image_id)#jsonファイルに記録
        
        self.unbind()
        self.data_clear()
        self.counter_img += 1
        self.cap = cv2.VideoCapture('C:/Users/naoki/anaconda_work/Faster-R-CNN-pytorch-main/data/ori_data/drama_clip/{}/clip.mp4'.format(self.counter_img - 1))
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.play_video()
        
    def data_clear(self):
        
        for i in range(self.counter_box+1):
            
            if self.rect_list[i] == None:
                pass
            else:
                self.canvas.delete(self.rect_list[i])
                self.rect_id_list[i].destroy()
                self.label_list[i].destroy()
                self.risk_label_list[i].destroy()
                
                self.risk_priority_list[i].destroy()
                
                self.txt_list[i].destroy()
                self.button_list[i].destroy()
                self.anno_canvas.delete("frame_{}".format(i))
                for item in range(len(self.item)):
                    self.check_list[i][item].destroy()
                for item in range(len(self.object)):
                    self.radio_list[i][item].destroy()
                

        self.remove_counter = 0
        self.counter_box = -1
        
        self.rect_list = []
        self.txt_list = []
        self.button_list = []
        self.txt_var_list = []
        self.check_list = []
        self.radio_list = []
        self.label_list = []
        

        self.risk_label_list = []
        
        self.risk_priority_list = []
        
 
        
        self.rect_id_list = []
        self.frame_list = []
        self.check_var_list = []
        self.radio_var_list = []
        self.all_check_var_list = []
        self.all_radio_var_list = []
        


        
        
        self.label_pos_list = []
        self.button_pos_list = []
        self.text_pos_list = []
        self.check_pos_list = []
        self.radio_pos_list = []
        self.risk_label_pos_list = []
        
        self.risk_priority_pos_list = []
        self.frame_pos_list = []
        

        
        self.box_start_list = []
        self.box_end_list = []

        self.check_var_dict = {}
        self.txt_var_list = []
        
    def data_hidden(self, button_num):
        
        None_remove_list = [i for i in range(len(self.label_list)) if self.label_list[i] is not None]
        #print(None_remove_list[3:])
        if len(None_remove_list[3:]) == 0:
            two_page_label = 1
        else:
            two_page_label = max(None_remove_list[3:])
        while len(None_remove_list) < 10:
            None_remove_list.append(None)
        
        one_page_label = None_remove_list[2]
        
        
        #print(one_page_label, two_page_label)
        if button_num == 2 and one_page_label != None:
            for i in range(one_page_label+1):
                if self.label_list[i] != None:
                    self.label_list[i].place_forget()
                    self.risk_label_list[i].place_forget()
                    self.risk_priority_list[i].place_forget()
                    self.txt_list[i].place_forget()
                    self.button_list[i].place_forget()
                    self.anno_canvas.itemconfigure("frame_{}".format(i), state='hidden')
                
                    for item in range(len(self.item)):
                        self.check_list[i][item].place_forget()
                    for item in range(len(self.object)):
                        self.radio_list[i][item].place_forget()
                        
            for i in range(one_page_label+1,len(self.label_list)):
                if self.label_list[i] != None:
                    
                    self.button_list[i].place(x=self.button_pos_list[i][0],y=self.button_pos_list[i][1]-self.one_space*3)
                    
                    self.txt_list[i].place(x=self.text_pos_list[i][0],y=self.text_pos_list[i][1]-self.one_space*3)
                    
                    self.label_list[i].place(x=self.label_pos_list[i][0],y=self.label_pos_list[i][1]-self.one_space*3)
                    
                    self.risk_label_list[i].place(x=self.risk_label_pos_list[i][0],y=self.risk_label_pos_list[i][1]-self.one_space*3)
                    
                    self.risk_priority_list[i].place(x=self.risk_priority_pos_list[i][0],y=self.risk_priority_pos_list[i][1]-self.one_space*3)
                    
                    self.anno_canvas.itemconfigure("frame_{}".format(i), state='normal')
                    self.anno_canvas.move("frame_{}".format(i), 0, -self.one_space*3)
                    
                    for l in range(len(self.item)):
                        #print(self.check_list[i][l])
                        
                        self.check_list[i][l].place(x=self.check_pos_list[i][l][0],y=self.check_pos_list[i][l][1]-self.one_space*3)

                    
                    for l in range(len(self.object)):
                        #print(self.check_list[i][l])
                        
                        self.radio_list[i][l].place(x=self.radio_pos_list[i][l][0],y=self.radio_pos_list[i][l][1]-self.one_space*3)
                        
        elif button_num == 1:
            
            for i in range(one_page_label+1,len(self.label_list)):
                self.anno_canvas.move("frame_{}".format(i), 0, +self.one_space*3)
            
            for i in range(one_page_label+1,two_page_label+1):
                if self.label_list[i] != None:
                    self.label_list[i].place_forget()
                    self.risk_label_list[i].place_forget()
                    self.risk_priority_list[i].place_forget()
                    self.txt_list[i].place_forget()
                    self.button_list[i].place_forget()
                    self.anno_canvas.itemconfigure("frame_{}".format(i), state='hidden')
                
                    for item in range(len(self.item)):
                        self.check_list[i][item].place_forget()
                    for item in range(len(self.object)):
                        self.radio_list[i][item].place_forget()
                        
            for i in range(0,one_page_label+1):
                if self.label_list[i] != None:
                    
                    self.button_list[i].place(x=self.button_pos_list[i][0],y=self.button_pos_list[i][1])
                    
                    self.txt_list[i].place(x=self.text_pos_list[i][0],y=self.text_pos_list[i][1])
                    
                    self.label_list[i].place(x=self.label_pos_list[i][0],y=self.label_pos_list[i][1])
                    
                    self.risk_label_list[i].place(x=self.risk_label_pos_list[i][0],y=self.risk_label_pos_list[i][1])
                    
                    self.risk_priority_list[i].place(x=self.risk_priority_pos_list[i][0],y=self.risk_priority_pos_list[i][1])
                    
                    self.anno_canvas.itemconfigure("frame_{}".format(i), state='normal')
                    #self.anno_canvas.move("frame_{}".format(i), 0, -self.one_space)
                    
                    for l in range(len(self.item)):
                        #print(self.check_list[i][l])
                        
                        self.check_list[i][l].place(x=self.check_pos_list[i][l][0],y=self.check_pos_list[i][l][1])

                    
                    for l in range(len(self.object)):
                        #print(self.check_list[i][l])
                        
                        self.radio_list[i][l].place(x=self.radio_pos_list[i][l][0],y=self.radio_pos_list[i][l][1])

    def disable_checkboxes(self, counter_box):
        def x():
            self.all_enable_checkboxes(counter_box)
        
            confirm_radio=self.all_radio_var_list[counter_box].get()
            #print(confirm_radio)
            if confirm_radio == 0:
                for i in range(1,7):
                    #print(i)
                    self.check_list[counter_box][i].config(state=tk.DISABLED)
                    self.check_list[counter_box][i].config(bg="black")
                    self.check_list[counter_box][i].deselect()
                    
            if confirm_radio == 1 or confirm_radio == 2:
                self.check_list[counter_box][0].config(state=tk.DISABLED)
                self.check_list[counter_box][0].config(bg="black")
                self.check_list[counter_box][0].deselect()
                
            if confirm_radio == 3:
                for i in range(0,7):
                    self.check_list[counter_box][i].config(state=tk.DISABLED)
                    self.check_list[counter_box][i].config(bg="black")
                    self.check_list[counter_box][i].deselect()
                    
        return x
    
    def all_enable_checkboxes(self, counter_box):
        
        for i in range(len(self.item)):
            self.check_list[counter_box][i].config(bg=self.bg_color[i])
            self.check_list[counter_box][i].config(state=tk.NORMAL)

    def btn_click_re(self):
        self.unbind()
        self.hidden_id()
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.play_video()
    
    def hidden_id(self):
        for i in range(len(self.rect_id_list)):
            if self.rect_id_list[i] != None:
                self.rect_id_list[i].place_forget()
    
    def save_json(self, s_rect, e_rect, text, risk, obj, img_id): 
        
        """
        
        btn_click_next()関数から呼び出され、入力されたデータを加工しjsonファイルとして保存する
        
        args
        ---------------
        
        s_rect : list
            各ボックスの左上の座標が格納されたリスト
        e_rect : list
            各ボックスの右下の座標が格納されたリスト
        text : list
            各ボックスのテキストバックスに打ち込まれた文字列に関するオブジェクトが格納されたリスト
        risk_degree : list
            各ボックスに保持されているチェックボックスのチェック可否の情報が含まれるオブジェクトが格納されたリスト
        img_id : int
            画面に表示されている画像の画像ID
        
        """
        
        self.one_region_data = {"regions":[]}

        record_s_rect_data = list(filter(None, s_rect))
        record_e_rect_data = list(filter(None, e_rect))
        record_txt_data = list(filter(None, text))
        record_risk_data = list(filter(None, risk))
        record_obj_data = list(filter(None, obj))
        
        all_box_risk_data = []
        #print(record_obj_data)
        
        for i in range(len(record_txt_data)):
            record_txt_data[i] = record_txt_data[i].get()
            record_obj_data[i] = record_obj_data[i].get()
            if record_obj_data[i] == 0:
                record_obj_data[i] = "歩行者"
            elif record_obj_data[i] == 1:
                record_obj_data[i] = "四輪車"
            elif record_obj_data[i] == 2:
                record_obj_data[i] = "モーターバイク"
            elif record_obj_data[i] == 3:
                record_obj_data[i] = "自転車"
                
            one_risk_data = []

            for item in range(len(self.item)):
                one_risk_data.append(1 if record_risk_data[i][item].get()==True else 0)
            
            

            all_box_risk_data.append(one_risk_data)
            
        
        #print(all_box_stage_data)        

        for i in range(len(record_s_rect_data)):
            self.region_id += 1
            self.one_region_data["regions"].append({"region_id": self.region_id, 
                                                    "boxes":[record_s_rect_data[i][0], record_s_rect_data[i][1],record_e_rect_data[i][0], record_e_rect_data[i][1]], 
                                                    "image_id": int(img_id), "ori_Danger_item": record_txt_data[i], "object": record_obj_data[i], "Danger_item": str(all_box_risk_data[i])})
            
            
        self.one_region_data["id"] = int(img_id)
        
        self.region_json_data.append(self.one_region_data)
        
        self.image_json_data.append({"width": self.img_width, "url": "ori_data/{}.jpg".format(img_id), "height": self.img_height, "image_id": int(img_id)})
        
        
        with open(self.ori_data_path + 'region_danger_info_data.json', 'w') as fp:
            json.dump(self.region_json_data, fp, ensure_ascii=False, indent=2)
            
        with open(self.ori_data_path + 'img_info_data.json', 'w') as fp:
            json.dump(self.image_json_data, fp, ensure_ascii=False, indent=2)
        
        with open('class_label.json', 'w') as fp:
            json.dump(self.item, fp, ensure_ascii=False)
        
        
def main():
    
    root = tk.Tk()
    app = Application(master=root)
    app.mainloop()

if __name__ == "__main__":
    main()

a
a


In [132]:
import tkinter as tk

def button_clicked():
    print("ボタンがクリックされました！")

root = tk.Tk()

button = tk.Button(root, text="ボタン", command=button_clicked)
button.pack()

# ボタンをコードで押す
button.invoke()

root.mainloop()

ボタンがクリックされました！
ボタンがクリックされました！


In [24]:
for i in range(5,6):
    print(i)

5


In [171]:
import tkinter as tk

def disable_checkboxes():
    checkbox2.config(state=tk.DISABLED)
    checkbox3.config(state=tk.DISABLED)

def enable_checkboxes():
    checkbox2.config(state=tk.NORMAL)
    checkbox3.config(state=tk.NORMAL)

root = tk.Tk()

radio_button_var = tk.IntVar()
radio_button1 = tk.Radiobutton(root, text="Radio 1", variable=radio_button_var, value=1,
                              command=disable_checkboxes)
radio_button2 = tk.Radiobutton(root, text="Radio 2", variable=radio_button_var, value=2,
                              command=enable_checkboxes)

checkbox1 = tk.Checkbutton(root, text="Checkbox 1")
checkbox2 = tk.Checkbutton(root, text="Checkbox 2")
checkbox3 = tk.Checkbutton(root, text="Checkbox 3")

radio_button1.pack()
radio_button2.pack()
checkbox1.pack()
checkbox2.pack()
checkbox3.pack()

root.mainloop()

In [204]:
import tkinter as tk

def deselect_checkbox():
    checkbox.deselect()

root = tk.Tk()

checkbox = tk.Checkbutton(root, text="Checkbox")
checkbox.pack()

deselect_button = tk.Button(root, text="Deselect", command=deselect_checkbox)
deselect_button.pack()

root.mainloop()

In [18]:
import tkinter as tk

def on_mousewheel(event):
    canvas.yview_scroll(-1 * int((event.delta / 120)), "units")

root = tk.Tk()
root.title("Scrollable Frame Example")

# スクロールバーの作成
scrollbar = tk.Scrollbar(root)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# キャンバスの作成
canvas = tk.Canvas(root, yscrollcommand=scrollbar.set)
canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# スクロールバーとキャンバスの連携
scrollbar.config(command=canvas.yview)

# フレームの作成
frame = tk.Frame(canvas)

# フレームにウィジェットを追加
# ここに必要なウィジェットを追加してください

# キャンバスにフレームを配置
canvas.create_window((0, 0), window=frame, anchor=tk.NW)

# マウスホイールでスクロール
canvas.bind_all("<MouseWheel>", on_mousewheel)

root.mainloop()

In [35]:
import tkinter as tk

def main():
    root = tk.Tk()
    root.geometry("800x600")

    frame2 = tk.Frame(root, width=640, height=925, relief=tk.RAISED, bd=5, bg="lightgrey")
    frame2.place(x=1025, y=20)

    scrollbar = tk.Scrollbar(frame2)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    anno_canvas = tk.Canvas(frame2, width=640, height=925, yscrollcommand=scrollbar.set)
    anno_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    scrollbar.config(command=anno_canvas.yview)

    # キャンバス内に要素を追加する場合
    anno_canvas.create_text(320, 20, text="Scrollable Canvas", font=("Arial", 16), anchor=tk.CENTER)

    # マウスホイールによるスクロールの設定
    anno_canvas.bind_all("<MouseWheel>", lambda event: anno_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units"))

    root.mainloop()

if __name__ == "__main__":
    main()


In [38]:
import tkinter as tk

def button_clicked(button_num):
    # ボタンの状態を変更する
    if button_num == 2:
        button2.config(state=tk.DISABLED)  # ボタン2を無効化
        # 他のボタンの状態を有効化する場合は以下のように記述します
        # button1.config(state=tk.NORMAL)
        # button3.config(state=tk.NORMAL)

root = tk.Tk()

# ボタンの作成
button1 = tk.Button(root, text="ボタン1", command=lambda: button_clicked(1))
button2 = tk.Button(root, text="ボタン2", command=lambda: button_clicked(2))
button3 = tk.Button(root, text="ボタン3", command=lambda: button_clicked(3))

# ボタンの配置
button1.pack()
button2.pack()
button3.pack()

root.mainloop()


In [7]:
data = [1,1,None,1,None]

a = [i for i in range(len(data)) if data[i] is not None]
a

[0, 1, 3]

In [83]:
a[2:]

[3]

In [114]:
my_list = [2, 3, None, None]

# 最も左のNoneのインデックスを取得
none_index = my_list.index(None)

# none_indexよりも前の整数を取得
left_integer = my_list[none_index - 1]

print(left_integer)

3
